In [43]:
import os
import itertools
import subprocess

class ChlamySimulationRunner:
    def __init__(self, 
                 output_dir="output_file", 
                 log_dir="logs", 
                 param_dir="params"):

        self.output_dir = output_dir
        self.log_dir = log_dir
        self.param_dir = param_dir

        os.makedirs(self.output_dir, exist_ok=True)
        os.makedirs(self.log_dir, exist_ok=True)
        os.makedirs(self.param_dir, exist_ok=True)

        self.Ns = [int(1e6)]
        self.taus = [1e-3]
        self.Rs = [5e-6]
        self.Hs = [100e-6]
        self.eta_0s = [1e-3]
        self.kbTs = [2.35e-16] #300 * 1.380649e-23]
        self.vs = [00e-6]
        self.Ls = [88e-3]
        self.Ws = [180e-6]
        self.Delta_Ps = [1e3]
        self.Ts = [2.]
        self.n_taskss = [1]
        self.N_trajectoriess = [10]
        self.oscillations = True # U can change this and pass it to run. too tired to do it
        
    def generate_param_file(self, filepath, **params):
        with open(filepath, "w") as f:
            for key, value in params.items():
                f.write(f"{key}={value}\n")

    def build_command(self, output_path, param_dict):
        base_cmd = [
            "python", "chlamy_run_simul.py",
            "-o", output_path
        ]
        for key, value in param_dict.items():
            flag = f"--{key}" if key not in ["N", "tau", "R", "H", "eta_0", "kbT", "v", "L", "W", "Delta_P", "T"] else f"-{key}"
            base_cmd.extend([flag, str(value)])
        return base_cmd

    def run_all(self):
        for N, tau, R, H, eta_0, kbT, v, L, W, Delta_P, T, n_tasks, N_trajectories in itertools.product(
            self.Ns, self.taus, self.Rs, self.Hs, self.eta_0s, self.kbTs,
            self.vs, self.Ls, self.Ws, self.Delta_Ps, self.Ts, self.n_taskss, self.N_trajectoriess
        ):
            param_dict = {
                "N": N, "tau": tau, "R": R, "H": H, "eta_0": eta_0, "kbT": kbT,
                "v": v, "L": L, "W": W, "Delta_P": Delta_P, "T": T,
                "n_tasks": n_tasks, "N_trajectories": N_trajectories
            }

            filename_base = f"{N}_{tau}_{R}_{H}_{eta_0}_{kbT}_{v}_{T}_{Delta_P}_{L}_{W}_{n_tasks}_{N_trajectories}"
            output_path = self.output_dir #os.path.join(self.output_dir, filename_base)
            param_path = os.path.join(self.param_dir, filename_base + ".txt")
            log_path = os.path.join(self.log_dir, filename_base + ".out")

            self.generate_param_file(param_path, **param_dict)
            command = self.build_command(output_path, param_dict)

            with open(log_path, "w") as log_file:
                process = subprocess.run(command, stdout=log_file, stderr=subprocess.STDOUT)
                if process.returncode != 0:
                    print(f"Simulation failed for: {filename_base}. See log: {log_path}")


# Create an instance of the simulations to import parameters

In [44]:
import numpy as np


In [45]:
runner = ChlamySimulationRunner()


### Pressure array, log spaced, to match Brady plot velocities

In [46]:
pressure_array_logspaced = np.logspace(
        np.log(.1) / np.log(10), # Brady velodities
        np.log(1000.) / np.log(10),
        10,
        base=10, )

pressure_array_logspaced = np.append(pressure_array_logspaced, 0)

In [47]:
pressure_array_logspaced

array([1.00000000e-01, 2.78255940e-01, 7.74263683e-01, 2.15443469e+00,
       5.99484250e+00, 1.66810054e+01, 4.64158883e+01, 1.29154967e+02,
       3.59381366e+02, 1.00000000e+03, 0.00000000e+00])

### Run all the simulations for the Brady plot

In [48]:
Brady_plot_runner = ChlamySimulationRunner()
Brady_plot_runner.output_dir = "./output_file/Brady_plot_oscillflow_passive"
Brady_plot_runner.Delta_Ps = pressure_array_logspaced
Brady_plot_runner.Ts = [1., 2., 4., 8.]
Brady_plot_runner.run_all()